In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [3]:
df=pd.read_csv('anime-dataset-2023.csv')

In [4]:
df

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",...,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,UNKNOWN,24723,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",...,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1931/...


In [5]:
df.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

# Some Minor Data Preprocessing


In [7]:
# Fill NA values with 0s
df = df.fillna(0)

# Filter rows containing "UNKNOWN" in any column
df = df[~df.isin(['UNKNOWN']).any(axis=1)]

# Extracting the first genre from the 'Genres' column
df['Genres'] = df['Genres'].str.split(',').str[0]

# Function to extract the starting year
def extract_start_year(aired):
    if 'to' in aired:
        return int(aired.split(', ')[1].split(' ')[0])
    else:
        return int(aired.split(', ')[1])

# Apply the function to the "Aired" column and create the "Release" column
df['Release'] = df['Aired'].apply(extract_start_year)

# Filter the dataset based on genres and remove specified columns
filtered_anime = df[df['Genres'].str.contains('Comedy|Fantasy|Slice of Life')].drop(columns=['Other name', 'Synopsis'])

In [8]:
# Converts the DataFrame 'df' into an interactive DataFrame 'idf' for dynamic data visualization and manipulation.
idf=df.interactive()

# Favorites and Popularity over Release year by Sources

In [10]:
# Creates an integer slider widget for selecting a year between 1960 and 2025, starting at 2000.
year_slider=pn.widgets.IntSlider(name='Year slider',start=1960,end=2025,step=1,value=2000)
year_slider

IntSlider(end=2025, name='Year slider', start=1960, value=2000)

In [11]:
# Creates a radio button group widget named 'Y axis' with options for 'Favorites' and 'Popularity'
yaxis_anime=pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['Favorites','Popularity',],
    button_type='success'
)

In [12]:
# Gets unique values from the 'Source' column of 'df' and prints them
unique_values = df['Source'].unique()
print(unique_values)

['Original' 'Manga' '4-koma manga' 'Light novel' 'Visual novel' 'Novel'
 'Other' 'Game' 'Unknown' 'Card game' 'Book' 'Radio' 'Mixed media'
 'Web manga' 'Music']


In [13]:
# Gets unique values from the 'Genres' column of 'df' and prints them
unique_values = df['Genres'].unique()
print(unique_values)

['Action' 'Adventure' 'Comedy' 'Drama' 'Sports' 'Award Winning' 'Sci-Fi'
 'Horror' 'Boys Love' 'Ecchi' 'Slice of Life' 'Fantasy' 'Avant Garde'
 'Mystery' 'Supernatural' 'Suspense' 'Romance' 'Gourmet' 'Girls Love']


### (1) Line Chart - Favorites and Popularity over release year by Sources

In [15]:
Sources = ['Manga', 'Light novel', 'Novel', 'Other']  # List of valid sources

anime_pipeline = (  # Filters, groups, and processes the interactive DataFrame 'idf' based on the conditions and selected y-axis metric
    idf[
        (idf.Release <= year_slider) &  # Filters rows where 'Release' is less than or equal to the slider value
        (idf.Source.isin(Sources))      # Filters rows where 'Source' is in the specified list of sources
    ]
    .groupby(['Source', 'Release'])[yaxis_anime].mean()  # Groups by 'Source' and 'Release', calculates the mean of the selected y-axis metric
    .to_frame()  # Converts the Series to a DataFrame
    .reset_index()  # Resets the index of the DataFrame
    .sort_values(by='Release')  # Sorts the DataFrame by 'Release' column
    .reset_index(drop=True)  # Resets the index again, dropping the old index
)


In [16]:
anime_pipeline

In [17]:
anime_plot = anime_pipeline.hvplot(x='Release', by='Source', y=yaxis_anime, line_width=2, title="Favorites and Popularity over Release year by Sources")  # Creates a plot of anime scores by release year and source with specified line width and title
anime_plot  # Displays the plot

### (2) Table - Favorites and Popularity over release year by Sources

In [19]:
anime_table = anime_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')  # Creates a paginated, stretchable table widget from 'anime_pipeline' with 10 rows per page
anime_table  # Displays the table

### (3) Scatterplot - Scored By and Members over Release year by Sources

In [21]:
def format_members(value):
    if value >= 1_000_000:
        return f"{value/1_000_000:.1f}M"
    elif value >= 1_000:
        return f"{value/1_000:.1f}K"
    else:
        return str(value)

score_vs__members_pipeline = (
    idf[
        (idf.Release <= year_slider) & 
        (idf.Source.isin(Sources))
    ]
    .groupby(['Source', 'Release', 'Score'])['Members']
    .mean()
    .to_frame()
    .reset_index()
    .assign(Members=lambda x: x['Members'].apply(format_members))  # Format the Members column
    .sort_values(by='Release')
    .reset_index(drop=True)
)

In [22]:
score_vs__members_pipeline

In [23]:
score_vs_members_scatterplot = score_vs__members_pipeline.hvplot(x='Score',
                                                               y = 'Members',
                                                               by = 'Source',
                                                               size = 79,
                                                               kind = 'scatter',
                                                               alpha = 0.7,
                                                               legend = False,
                                                               height = 500,
                                                               width = 500)

In [24]:
score_vs_members_scatterplot

### (4) Bar Chart - Popularity and Favorites over Release year by Genres

In [26]:
yaxis_anime_genres = pn.widgets.RadioButtonGroup(name='Y axis', options=['Popularity', 'Favorites'], button_type='success')  # Creates a radio button group widget for selecting the y-axis metric

genres_excl_world = ['Action','Adventure','Comedy','Drama','Sports','Award Winning','Sci-Fi','Horror','Boys Love','Ecchi','Slice of Life','Fantasy','Avant Garde','Mystery','Supernatural','Suspense','Romance','Gourmet','Girls Love'] # List of sources excluding 'World'

anime_genres_bar_pipeline = (  # Processes 'idf' for the specified conditions and selected y-axis metric
    idf[
        (idf.Release == year_slider) &  # Filters rows where 'Release' equals the slider value
        (idf.Genres.isin(genres_excl_world))  # Filters rows where 'Genres'
    ]
    .groupby(['Release', 'Genres'])[yaxis_anime_genres].sum()  # Groups by 'Release' and 'Genres', calculates the sum of the selected y-axis metric
    .to_frame()  # Converts the Series to a DataFrame
    .reset_index()  # Resets the index of the DataFrame
    .sort_values(by='Release')  # Sorts the DataFrame by 'Release' column
    .reset_index(drop=True)  # Resets the index again, dropping the old index
)

In [27]:
anime_genres_bar_plot = anime_genres_bar_pipeline.hvplot(kind='bar',  # Creates a bar plot of anime releases grouped by genres with specified settings
                                                         x='Genres',
                                                         y=yaxis_anime_genres,
                                                         title='Popular and favorites anime release by genres')
anime_genres_bar_plot  # Displays the bar plot

In [28]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Anime Dashboard', 
    sidebar=[pn.pane.Markdown("# Stats of Anime"), 
             pn.pane.Markdown("#### Anime is a style of animation popular in Japanese films and television series. It often combines stark, colorful graphics with action-packed plots. Early anime films were intended primarily for a Japanese audience. Therefore, they used many cultural references unique to Japan."), 
             pn.pane.PNG('Kakashi.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_anime, 
                           anime_plot.panel(width=700), margin=(0,25)), 
                 anime_table.panel(width=500)), 
          pn.Row(pn.Column(score_vs_members_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_anime_genres, anime_genres_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();